Compare a simulation with and without backreaction. You need to run the same setup, only with this difference. You also need to save "data", "bilinears", and "quantum" (this is what the script includes, you can remove something if you prefer). You could also compare "correlators".

In [ ]:
using HDF5
using LaTeXStrings
using Plots ; pythonplot()
using DelimitedFiles
using SpheriCo

In [ ]:
# give the directory where the data from all the runs are saved
dir = "../examples/quantum_runs/"
par1 = "a0.4_b0.0_c4.0_rmax15.0_tmax7.0_cfl0.0625_sigma0.0_overMp2_25.132741228718345_reg_true_backreact_true_rcut12.0_mPV1.0_dk_denom_15_kmax10.0_lmax30.0"
par2 = "a0.4_b0.0_c4.0_rmax15.0_tmax7.0_cfl0.0625_sigma0.0_overMp2_25.132741228718345_reg_true_backreact_false_mPV1.0_dk_denom_15_kmax10.0_lmax30.0"

your_dir1 = dir*par1
your_dir2 = dir*par2

# set manually, match the one of the simulation
mPV = 1.0
CC = -log(3^9/2^16)*(mPV^4)/(8*(2*π)^2);
#println("CC=",CC)

# convention
hbar = 1.0;
c = 1.0;

In [ ]:
# number of radial grid points
D = 3
Nr = 128*2^D + 3 # the overal course graining

# load the r grid
r = h5read(your_dir1*"/data_$(Nr)/r.h5","r")
dr = r[2] - r[1]
# needed later
id = ones(length(r));

# list all available iterations (and corresponding files)
(its, all_filenames)           = list_h5_files(your_dir1*"/data_$(Nr)", prefix="data_");
(its_bln, all_filenames_bln)   = list_h5_files(your_dir1*"/data_$(Nr)", prefix="bilinears_");
(its_quantum, all_filenames_quantum)   = list_h5_files(your_dir1*"/data_$(Nr)", prefix="quantum_");

# list of iterations with saved data for the different data-types
println("length(its)=",length(its))
println("its",its)

println("length(its_bln)=",length(its_bln))
println("its_bln=",its_bln)

println("length(its_quantum)=",length(its_quantum))
println("its_quantum=",its_quantum)

The filter below is used to reconstruct the filter in the backreaction case. Set r_cut and n manually to the values used in the simulation. Needed later.

In [ ]:
r_cut = 12
Nr = length(r)
n = 1.5
filter = zeros(Nr)
filter[3:end] = 0.5*ones(Nr-2) + 0.5*tanh.(-r[3:end].^n .+ (r_cut^n).*ones(Nr-2))
filter[2]=filter[4]
filter[1]=filter[5];

In [ ]:
# iteration when you want to do the comparison
it = 50

it_data = its[it]
it_str_data  = lpad(it_data, 4, "0")

it_bln = its_bln[it]
it_str_bln  = lpad(it_bln, 4, "0")

it_quantum = its_quantum[it]
it_str_quantum  = lpad(it_quantum, 4, "0")

v_classic_labels = ["Φ", "Π", "Ψ", "A", "B", "DB", "Utld", "K", "KB", "λ", "α", "Dα", "Θ", "Zr", "f", "g", "U", "V"]
# dir 1
v1 = h5read(your_dir1*"/data_$(Nr)/data_$(it_str_data).h5","v")
Π1  = v1[:,2]
Ψ1  = v1[:,3]
A1  = v1[:,4]
oA1 = 1.0./A1
B1  = v1[:,5]
oB1 = 1.0./B1
KB1 = v1[:,9]
α1  = v1[:,11]
K1  = v1[:,8]

bln1  = h5read(your_dir1*"/data_$(Nr)/bilinears_$(it_str_bln).h5","bilinears")
v_quantum1 = h5read(your_dir1*"/data_$(Nr)/quantum_$(it_str_quantum).h5","v")

# if there is an error when loading the time attribute, try to change "./" to "." or to "/" 
t_data1    =  h5readattr(your_dir1*"/data_$(Nr)/data_$(it_str_data).h5", "/")["time"]
t_bln1     =  h5readattr(your_dir1*"/data_$(Nr)/bilinears_$(it_str_bln).h5", "/")["time"]
t_quantum1 =  h5readattr(your_dir1*"/data_$(Nr)/quantum_$(it_str_quantum).h5", "/")["time"]

# dir 2
v2 = h5read(your_dir2*"/data_$(Nr)/data_$(it_str_data).h5","v")
Π2  = v2[:,2]
Ψ2  = v2[:,3]
A2  = v2[:,4]
oA2 = 1.0./A2
B2  = v2[:,5]
oB2 = 1.0./B2
KB2 = v2[:,9]
α2  = v2[:,11]
K2  = v2[:,8]

bln2  = h5read(your_dir2*"/data_$(Nr)/bilinears_$(it_str_bln).h5","bilinears")
v_quantum2 = h5read(your_dir2*"/data_$(Nr)/quantum_$(it_str_quantum).h5","v")

# if there is an error when loading the time attribute, try to change "./" to "." or to "/" 
t_data2    =  h5readattr(your_dir2*"/data_$(Nr)/data_$(it_str_data).h5", "/")["time"]
t_bln2     =  h5readattr(your_dir2*"/data_$(Nr)/bilinears_$(it_str_bln).h5", "/")["time"]
t_quantum2 =  h5readattr(your_dir2*"/data_$(Nr)/quantum_$(it_str_quantum).h5", "/")["time"]

# checks if time instant is the same for all types of data
@assert t_data1 ≈ t_bln1 ≈ t_quantum1 ≈ t_data2 ≈ t_bln2 ≈ t_quantum2

#dir 1
# classical
ρ1  = 0.5.*oA1.*((oB1.^2.0).*(Π1.^2.0) .+ Ψ1.^2.0)
jA1 = -(oA1.^0.5).*oB1.*Π1.*Ψ1
SA1 = 0.5.*oA1.*((oB1.^2.0).*(Π1.^2.0) .+ Ψ1.^2.0)
SB1 = 0.5.*oA1.*((oB1.^2.0).*(Π1.^2.0) .- Ψ1.^2.0)

ρ_quantum1 = @. (hbar*c^2/(4.0*π))*((0.5/α1^2)*bln1[:,2] + (0.5/A1)*bln1[:,3] + (1/B1)*bln1[:,5] + 0.5*bln1[:,1] );
jA_quantum1 = @. -(hbar*c^2/(4.0*π))*(1.0/α1)*bln1[:,4];
SA_quantum1 = @. (hbar*c^2/(4.0*π))*( (0.5/α1^2)*bln1[:,2] + (0.5/A1)*bln1[:,3] - (1/B1)*bln1[:,5] - 0.5*bln1[:,1] );
SB_quantum1 = @. (hbar*c^2/(4.0*π))*( (0.5/α1^2)*bln1[:,2] -(0.5/A1)*bln1[:,3] - 0.5*bln1[:,1] );

ρ_full1 = ρ1 + real.(ρ_quantum1).*filter
SA_full1 = SA1 + real.(SA_quantum1).*filter
SB_full1 = SB1 + real.(SB_quantum1).*filter
jA_full1 = jA1 + real.(jA_quantum1).*filter

# matter content (sources) in K rhs ~ 0.5*SA + SB - Λ + 0.5*ρ
source_K1 = @. 0.5*SA_full1 + SB_full1 - CC*id*filter + 0.5*ρ_full1;
# matter content (sources) in KB rhs ~ SA - 2*Λ - ρ
source_KB1 = @. SA_full1 - 2.0*CC*id*filter - ρ_full1;

#dir 2
# classical
ρ2  = 0.5.*oA2.*((oB2.^2.0).*(Π2.^2.0) .+ Ψ2.^2.0)
jA2 = -(oA2.^0.5).*oB2.*Π2.*Ψ2
SA2 = 0.5.*oA2.*((oB2.^2.0).*(Π2.^2.0) .+ Ψ2.^2.0)
SB2 = 0.5.*oA2.*((oB2.^2.0).*(Π2.^2.0) .- Ψ2.^2.0)

ρ_quantum2 = @. (hbar*c^2/(4.0*π))*((0.5/α2^2)*bln2[:,2] + (0.5/A2)*bln2[:,3] + (1/B2)*bln2[:,5] + 0.5*bln2[:,1] );
jA_quantum2 = @. -(hbar*c^2/(4.0*π))*(1.0/α2)*bln2[:,4];
SA_quantum2 = @. (hbar*c^2/(4.0*π))*( (0.5/α2^2)*bln2[:,2] + (0.5/A2)*bln2[:,3] - (1/B2)*bln2[:,5] - 0.5*bln2[:,1] );
SB_quantum2 = @. (hbar*c^2/(4.0*π))*( (0.5/α2^2)*bln2[:,2] -(0.5/A2)*bln2[:,3] - 0.5*bln2[:,1] );

ρ_full2 = ρ2 + real.(ρ_quantum2)
SA_full2 = SA2 + real.(SA_quantum2)
SB_full2 = SB2 + real.(SB_quantum2)
jA_full2 = jA2 + real.(jA_quantum2)

# matter content (sources) in K rhs = α(0.5*SA + SB - Λ + 0.5*ρ)/Mp^2
source_K2 = 0.5*SA_full2 + SB_full2 + 0.5*ρ_full2;
# matter content (sources) in KB rhs = 0.5*α(SA - Λ - ρ)/Mp^2
source_KB2 = SA_full2 - ρ_full2;

Plot various quantities for comparison.

The stucture of the bilinears:

bilin[:,1] = (4*π)/(hbar*c^2)* [μ^2*<Φ^2>]_quantum = Sum_{kl} μ^2*(2l+1)*|utld_kl|^2;

bilin[:,2] = (4*π*α^2)/(hbar*c^2*A*B^2)* <Π^2>_quantum = Sum_{kl} dk*(2*l+1)*|∂_t utld_kl|^2

bilin[:,3] = (4*π)/(hbar*c^2)* <Ψ^2>_quantum = Sum_{kl} dk*(2l+1)*|∂_r utld_kl|^2

bilin[:,4] = (4*π*α)/(hbar*c^2*A^(1/2)*B)* <Π*Ψ>_quantum = Sum_{kl} dk*0.5*(2*l+1)[(∂_r utld_kl)*conj(∂_t utld_kl) + conj(∂_r utld_kl)(∂_t utld_kl)]

bilin[:,5] = (4*π)/(hbar*c^2)* [(1/r^2)*<∂_θ Φ^2>]_quantum = Sum_{kl} dk*0.5*l*(l+1)*(2l+1)*|r^(l-1)*u_kl|^2;

In [ ]:
ri_min = 3
ri_max = 1*512
bln_i = 4

blni1 = bln1[:,bln_i]
blni2 = bln2[:,bln_i]

plot(r[ri_min:ri_max], real.(blni1)[ri_min:ri_max], title = "t=$(t_bln1)",
     label = "Re(bln1[$(bln_i)])", linewidth = 2, frame = true, wsize = (800,400), legend=:bottomright)

plot!(r[ri_min:ri_max], real.(blni2)[ri_min:ri_max],
     label = "Re(bln2[$(bln_i)])", linewidth = 2, frame = true)


In [ ]:
ri_min = 3
ri_max = 512*1
# choose l and k to choose a specific quantum mode
ll = 5.0
kk = 5.0
# choose the mass index 
mi = 1
# choose the function index (1 is uq, 2 is ψq, and 3 is πq
fi = 1

f1 = real.((v_quantum1)[:,Int(ll)+1,Int(kk),mi,fi]) 
f2 = real.((v_quantum2)[:,Int(ll)+1,Int(kk),mi,fi]) 

plot(r[ri_min:ri_max], f1[ri_min:ri_max], title = "t=$(t_bln1)",
     label = "f1", linewidth = 2, frame = true, wsize = (800,400))

plot!(r[ri_min:ri_max], f2[ri_min:ri_max],
     label = "f2", linewidth = 2)


In [ ]:
ri_min = 3
ri_max = 1*512

plot(r[ri_min:ri_max], (ρ_full1)[ri_min:ri_max], title = "t=$(t_bln1)",
     label = "ρ_full1", linewidth = 2, frame = true, wsize = (800,400), legend=:top)

plot!(r[ri_min:ri_max], (ρ_full2)[ri_min:ri_max],
     label = "ρ_full2", linewidth = 2)


In [ ]:
ri_min = 3
ri_max = 1*512

plot(r[ri_min:ri_max], (SA_full1)[ri_min:ri_max], title = "t=$(t_bln1)",
     label = "SA_full1", linewidth = 2, frame = true, wsize = (800,400), legend=:top)

plot!(r[ri_min:ri_max], (SA_full2)[ri_min:ri_max],
     label = "SA_full2", linewidth = 2)


In [ ]:
ri_min = 3
ri_max = 1*512

plot(r[ri_min:ri_max], (SB_full1)[ri_min:ri_max], title = "t=$(t_bln1)",
     label = "SB_full1", linewidth = 2, frame = true, wsize = (800,400), legend=:bottomright)

plot!(r[ri_min:ri_max], (SB_full2)[ri_min:ri_max], label = "SB_full2", linewidth = 2)

In [ ]:
ri_min = 3
ri_max = 1*512

plot(r[ri_min:ri_max], source_K1[ri_min:ri_max], title = "t=$(t_bln1)",
    label = "source_K1", linewidth = 2, frame = true, wsize = (800,400))

plot!(r[ri_min:ri_max], source_K2[ri_min:ri_max], label = "source_K2", linewidth = 2)

In [ ]:
ri_min = 3
ri_max = 1*512

plot(r[ri_min:ri_max], source_KB1[ri_min:ri_max], title = "t=$(t_bln1)",
     label = "source_KB1", linewidth = 2, frame = true, wsize = (800,400))

plot!(r[ri_min:ri_max], source_KB2[ri_min:ri_max], label = "source_KB2", linewidth = 2)

In [ ]:
ri_min = 3
ri_max = 1*512

plot(r[ri_min:ri_max], jA1[ri_min:ri_max], title = "t=$(t_bln1)",
     label = "jA1", linewidth = 2, frame = true, wsize = (800,400))

plot!(r[ri_min:ri_max], jA2[ri_min:ri_max], title = "t=$(t_bln2)",
     label = "jA2", linewidth = 2)

In [ ]:
ri_min = 3
ri_max = 2*512

plot(r[ri_min:ri_max], α1[ri_min:ri_max], title = "t=$(t_bln1)",
     label = "α1", linewidth = 2, frame = true, wsize = (800,400))

plot!(r[ri_min:ri_max], α2[ri_min:ri_max], label = "α2", linewidth = 2)

In [ ]:
ri_min = 3
ri_max = 2*512

plot(r[ri_min:ri_max], B1[ri_min:ri_max], title = "t=$(t_bln1)",
     label = "B1", linewidth = 2, frame = true, wsize = (800,400))

plot!(r[ri_min:ri_max], B2[ri_min:ri_max], label = "B2", linewidth = 2)


In [ ]:
ri_min = 3
ri_max = 2*512

plot(r[ri_min:ri_max], A1[ri_min:ri_max], title = "t=$(t_bln1)",
     label = "A1", linewidth = 2, frame = true, wsize = (800,400))

plot!(r[ri_min:ri_max], A2[ri_min:ri_max], label = "A2", linewidth = 2)


In [ ]:
ri_min = 3
ri_max = 2*512

plot(r[ri_min:ri_max], K1[ri_min:ri_max], title = "t=$(t_bln1)",
     label = "K1", linewidth = 2, frame = true, wsize = (800,400))

plot!(r[ri_min:ri_max], K2[ri_min:ri_max], label = "K2", linewidth = 2)


In [ ]:
ri_min = 3
ri_max = 2*512

plot(r[ri_min:ri_max], KB1[ri_min:ri_max], title = "t=$(t_bln1)",
     label = "KB1", linewidth = 2, frame = true, wsize = (800,400))

plot!(r[ri_min:ri_max], KB2[ri_min:ri_max], label = "KB2", linewidth = 2)
